# MD&A extraction from single report Jan 02
## Loading at least one part of the local data into google colab, extracting part of the MD&A report using python code

In [80]:
from google.colab import files

file_path = '/content/drive/MyDrive/Colab Notebooks/Julex/files/20200102/940944/0000940944-20-000003.txt'
with open(file_path, 'r') as file:
    content = file.read()

# Process the content of the file as needed
#for later user, would it be easier to loop over a bunch of file in the file_path ??? need to examine this


One way I thought about extracting as hinted by Professor Yeabin was to look for schedule. I extracted information between item 2 and item 3 which is the management discussion and analysis part, using re to initiate the search and compile the content until the stop word in start with item 3

In [81]:
import re #use regular expression to scrape the MD&A


with open(file_path, 'r') as file:
    content = file.read()

# Use regular expressions to extract the MD&A section based on a section heading pattern,
mda_match = re.search(r"(?i)\bitem\s*2\b.*?(?=\bitem\s*\d+\b|\Z)", content, re.DOTALL) #since this is 10-Q I use the sea
if mda_match:
    mda_text = mda_match.group()
    # Optionally, you can further process the extracted MD&A text

    # Print or use the extracted MD&A text
    print(mda_text)
else:
    print("MD&A section not found in the file")

Item 2.</span><span style="font-family:inherit;font-size:10pt;font-weight:bold;">Management&#8217;s Discussion and Analysis of Financial Condition and Results of Operations</span></div><div style="line-height:120%;padding-top:6px;text-indent:32px;font-size:10pt;"><span style="font-family:inherit;font-size:10pt;">The discussion and analysis below for the Company, which contains forward-looking statements, should be read in conjunction with the unaudited financial statements, the notes to such financial statements and the &#8220;Forward-Looking Statements&#8221; included elsewhere in this Form 10-Q. </span></div><div style="line-height:120%;padding-top:6px;text-indent:32px;font-size:10pt;"><span style="font-family:inherit;font-size:10pt;">To facilitate review of our discussion and analysis, the following table sets forth our financial results for the periods indicated.  All information is derived from the unaudited consolidated statements of earnings for the quarter and </span><span styl

However using package such as Beautiful soup helped us understand the content in format of text instead of HTML

In [82]:
from bs4 import BeautifulSoup

# Pattern to match the MD&A section, Same thing as used on above
pattern = r"(?i)\bitem\s*2\b.*?(?=\bitem\s*\d+\b|\Z)"

# Find the MD&A section in the content
mda_section = re.search(pattern, content, re.DOTALL)

if mda_section:
    # Extract the matched section
    mda_html = mda_section.group(0)

    # Parse the HTML using BeautifulSoup
    soup = BeautifulSoup(mda_html, 'html.parser')

    # Extract the plain text without HTML tags
    mda_text = soup.get_text()
    print(mda_text)
else:
    print("MD&A section not found in the HTML file")

Item 2.Management’s Discussion and Analysis of Financial Condition and Results of OperationsThe discussion and analysis below for the Company, which contains forward-looking statements, should be read in conjunction with the unaudited financial statements, the notes to such financial statements and the “Forward-Looking Statements” included elsewhere in this Form 10-Q. To facilitate review of our discussion and analysis, the following table sets forth our financial results for the periods indicated.  All information is derived from the unaudited consolidated statements of earnings for the quarter and six months ended November 24, 2019 and November 25, 2018.   Three Months Ended   Six Months Ended  (in millions)November 24, 2019 November 25, 2018 % Chg November 24, 2019 November 25, 2018 % ChgSales$2,056.4 $1,973.4 4.2 % $4,190.3 $4,034.8 3.9 %Costs and expenses:           Food and beverage583.0 563.3 3.5 1,186.3 1,146.6 3.5Restaurant labor692.3 662.4 4.5 1,396.1 1,341.7 4.1Restaurant ex

In [83]:
mda_text

"Item 2.Management’s Discussion and Analysis of Financial Condition and Results of OperationsThe discussion and analysis below for the Company, which contains forward-looking statements, should be read in conjunction with the unaudited financial statements, the notes to such financial statements and the “Forward-Looking Statements” included elsewhere in this Form 10-Q. To facilitate review of our discussion and analysis, the following table sets forth our financial results for the periods indicated.  All information is derived from the unaudited consolidated statements of earnings for the quarter and six months ended November\xa024, 2019 and November\xa025, 2018.\xa0 \xa0Three Months Ended\xa0\xa0\xa0Six Months Ended\xa0\xa0(in millions)November\xa024, 2019\xa0November\xa025, 2018\xa0% Chg\xa0November\xa024, 2019\xa0November\xa025, 2018\xa0% ChgSales$2,056.4\xa0$1,973.4\xa04.2\xa0%\xa0$4,190.3\xa0$4,034.8\xa03.9\xa0%Costs and expenses:\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0Food an

# Sentiment Analysis example
Here utilize the code from the Finbert ipynb document, since later will use larger data points, google colab would utilize GPU

Analyzing financial text sentiment is valuable as it can engage the views and opinions of managers, information intermediaries and investors. FinBERT-Sentiment is a FinBERT model fine-tuned on 10,000 manually annotated sentences from analyst reports of S&P 500 firms.

**Input**: A financial text.

**Output**: Positive, Neutral or Negative.

In [84]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [21]:
from transformers import BertTokenizer, BertForSequenceClassification, pipeline

In [22]:
# tested in transformers==4.18.0
import transformers
transformers.__version__

'4.30.2'

In [23]:
pip install torch torchvision torchaudio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [24]:
finbert = BertForSequenceClassification.from_pretrained('yiyanghkust/finbert-tone',num_labels=3)
tokenizer = BertTokenizer.from_pretrained('yiyanghkust/finbert-tone')

Early researches use BOW bag of word without looking at context and finbert instead take sentences as inputs to get a sense of sentiment.

Here I decided to use NLTK tokenzier to slice txt into separate strings.

In [61]:
import nltk #
nltk.download('punkt')

# Split the text into sentences
sentences = nltk.sent_tokenize(mda_text)

# Identify the starting and ending indices of the MD&A section
start_index = None
end_index = None
for i, sentence in enumerate(sentences):
    if 'Management’s Discussion and Analysis' in sentence: # slicing between the beginning of MDA to the end(which is item 3)
        start_index = i
    if 'Item 3.' in sentence:
        end_index = i
        break

# Extract the MD&A section
mda_sliced = ' '.join(sentences[start_index:end_index])

# Print the extracted MD&A section
print(mda_sliced)

Item 2.Management’s Discussion and Analysis of Financial Condition and Results of OperationsThe discussion and analysis below for the Company, which contains forward-looking statements, should be read in conjunction with the unaudited financial statements, the notes to such financial statements and the “Forward-Looking Statements” included elsewhere in this Form 10-Q. To facilitate review of our discussion and analysis, the following table sets forth our financial results for the periods indicated. All information is derived from the unaudited consolidated statements of earnings for the quarter and six months ended November 24, 2019 and November 25, 2018. Three Months Ended   Six Months Ended  (in millions)November 24, 2019 November 25, 2018 % Chg November 24, 2019 November 25, 2018 % ChgSales$2,056.4 $1,973.4 4.2 % $4,190.3 $4,034.8 3.9 %Costs and expenses:           Food and beverage583.0 563.3 3.5 1,186.3 1,146.6 3.5Restaurant labor692.3 662.4 4.5 1,396.1 1,341.7 4.1Restaurant expen

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [69]:
from nltk.tokenize import sent_tokenize

text = mda_sliced

sentences = sent_tokenize(text)
limited_sentences = []
remaining_chars = []

for sentence in sentences:
    if len(sentence) <= 512:    #the reason to put 512 is because Finbert only take 512 tokens as input one at a time
        limited_sentences.append(sentence)
    else:
        limited_sentences.append(sentence[:512])
        remaining_chars.append(sentence[512:])

print("Limited Sentences:")
print(limited_sentences)

print("Remaining Characters:")
print(remaining_chars)

Limited Sentences:
['Item 2.Management’s Discussion and Analysis of Financial Condition and Results of OperationsThe discussion and analysis below for the Company, which contains forward-looking statements, should be read in conjunction with the unaudited financial statements, the notes to such financial statements and the “Forward-Looking Statements” included elsewhere in this Form 10-Q.', 'To facilitate review of our discussion and analysis, the following table sets forth our financial results for the periods indicated.', 'All information is derived from the unaudited consolidated statements of earnings for the quarter and six months ended November\xa024, 2019 and November\xa025, 2018.', 'Three Months Ended\xa0\xa0\xa0Six Months Ended\xa0\xa0(in millions)November\xa024, 2019\xa0November\xa025, 2018\xa0% Chg\xa0November\xa024, 2019\xa0November\xa025, 2018\xa0% ChgSales$2,056.4\xa0$1,973.4\xa04.2\xa0%\xa0$4,190.3\xa0$4,034.8\xa03.9\xa0%Costs and expenses:\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa

In [70]:
nlp = pipeline("text-classification", model=finbert, tokenizer=tokenizer)
results = nlp(limited_sentences)

In [73]:
results

[{'label': 'Neutral', 'score': 0.999277651309967},
 {'label': 'Neutral', 'score': 0.9999874830245972},
 {'label': 'Neutral', 'score': 0.9999974966049194},
 {'label': 'Neutral', 'score': 0.9999767541885376},
 {'label': 'Neutral', 'score': 0.9959502220153809},
 {'label': 'Neutral', 'score': 0.9999983310699463},
 {'label': 'Neutral', 'score': 0.9999996423721313},
 {'label': 'Neutral', 'score': 0.9999933242797852},
 {'label': 'Neutral', 'score': 0.9997661709785461},
 {'label': 'Positive', 'score': 0.9999973773956299},
 {'label': 'Neutral', 'score': 0.9999507665634155},
 {'label': 'Neutral', 'score': 0.9999459981918335},
 {'label': 'Negative', 'score': 0.7479577660560608},
 {'label': 'Neutral', 'score': 0.9999078512191772},
 {'label': 'Neutral', 'score': 0.9999648332595825},
 {'label': 'Positive', 'score': 0.9999502897262573},
 {'label': 'Positive', 'score': 0.9999748468399048},
 {'label': 'Positive', 'score': 0.9999986886978149},
 {'label': 'Positive', 'score': 0.9999990463256836},
 {'labe

Neutral: 91 instances with scores ranging from 0.5916 to 0.9999
Positive: 12 instances with scores ranging from 0.6196 to 0.9999
Negative: 6 instances with scores ranging from 0.7479 to 0.9995

## ESG-Classification
ESG analysis can help investors determine a business' long-term sustainability and identify associated risks. FinBERT-ESG is a FinBERT model fine-tuned on 2,000 manually annotated sentences from firms' ESG reports and annual reports.

**Input**: A financial text.

**Output**: Environmental, Social, Governance or None.

In [86]:
finbert = BertForSequenceClassification.from_pretrained('yiyanghkust/finbert-esg',num_labels=4)
tokenizer = BertTokenizer.from_pretrained('yiyanghkust/finbert-esg')

In [89]:
nlp = pipeline("text-classification", model=finbert, tokenizer=tokenizer)
results_esg = nlp(limited_sentences)

In [90]:
results_esg

[{'label': 'None', 'score': 0.9947275519371033},
 {'label': 'None', 'score': 0.9955516457557678},
 {'label': 'None', 'score': 0.9964991807937622},
 {'label': 'None', 'score': 0.9948780536651611},
 {'label': 'None', 'score': 0.9955894947052002},
 {'label': 'None', 'score': 0.9960585832595825},
 {'label': 'None', 'score': 0.9802843928337097},
 {'label': 'None', 'score': 0.9958718419075012},
 {'label': 'None', 'score': 0.9949856996536255},
 {'label': 'None', 'score': 0.9960682392120361},
 {'label': 'None', 'score': 0.9955821633338928},
 {'label': 'None', 'score': 0.9955854415893555},
 {'label': 'None', 'score': 0.9940272569656372},
 {'label': 'None', 'score': 0.9891647696495056},
 {'label': 'None', 'score': 0.9939433932304382},
 {'label': 'None', 'score': 0.9957815408706665},
 {'label': 'None', 'score': 0.9957881569862366},
 {'label': 'None', 'score': 0.9961884617805481},
 {'label': 'None', 'score': 0.9957718253135681},
 {'label': 'None', 'score': 0.9957129955291748},
 {'label': 'None', '

## FLS-Classification
Forward-looking statements (FLS) inform investors of managers’ beliefs and opinions about firm's future events or results. Identifying forward-looking statements from corporate reports can assist investors in financial analysis. FinBERT-FLS is a FinBERT model fine-tuned on 3,500 manually annotated sentences from Management Discussion and Analysis section of annual reports of Russell 3000 firms.

**Input**: A financial text.

**Output**: Specific-FLS , Non-specific FLS, or Not-FLS.

In [92]:
finbert = BertForSequenceClassification.from_pretrained('yiyanghkust/finbert-fls',num_labels=3)
tokenizer = BertTokenizer.from_pretrained('yiyanghkust/finbert-fls')

In [93]:
nlp = pipeline("text-classification", model=finbert, tokenizer=tokenizer)
results = nlp(limited_sentences)

In [94]:
results

[{'label': 'Not FLS', 'score': 0.9829452633857727},
 {'label': 'Not FLS', 'score': 0.9927282333374023},
 {'label': 'Not FLS', 'score': 0.9932370781898499},
 {'label': 'Not FLS', 'score': 0.9846259951591492},
 {'label': 'Not FLS', 'score': 0.9909940361976624},
 {'label': 'Not FLS', 'score': 0.9903349280357361},
 {'label': 'Not FLS', 'score': 0.972093403339386},
 {'label': 'Not FLS', 'score': 0.9902895092964172},
 {'label': 'Not FLS', 'score': 0.9853202700614929},
 {'label': 'Not FLS', 'score': 0.9913359880447388},
 {'label': 'Not FLS', 'score': 0.9900984168052673},
 {'label': 'Not FLS', 'score': 0.9905932545661926},
 {'label': 'Specific FLS', 'score': 0.9139053225517273},
 {'label': 'Specific FLS', 'score': 0.8909835815429688},
 {'label': 'Not FLS', 'score': 0.9761738181114197},
 {'label': 'Not FLS', 'score': 0.9937041401863098},
 {'label': 'Not FLS', 'score': 0.9936489462852478},
 {'label': 'Not FLS', 'score': 0.993790328502655},
 {'label': 'Not FLS', 'score': 0.9932954907417297},
 {'l

The majority of the predictions indicate "Not FLS" with high scores: The model has assigned high scores to the label "Not FLS" for most of the financial text inputs. This suggests that the analyzed texts do not contain explicit forward-looking statements. Therefore, it appears that the texts predominantly consist of factual information, historical data, or other non-projection-related details.